In [132]:
import music_trees as mt
import glob

# let's look at which results we have available
result_csvs = glob.glob(str(mt.RESULTS_DIR / 'mdb'/ '*.csv'))
result_csvs

['/home/hugo/lab/music-trees/results/mdb/baseline-v0.csv',
 '/home/hugo/lab/music-trees/results/mdb/hproto-v1.csv',
 '/home/hugo/lab/music-trees/results/mdb/hproto-v2.csv']

In [133]:
import pandas as pd
dfs = [pd.read_csv(p) for p in result_csvs]
df = pd.concat(dfs, ignore_index=True)
df

,Unnamed: 0,episode_idx,metric,value,tag,model,n_shot,n_class,logger,checkpoint_callback,...,batch_size,num_workers,n_query,model_name,learning_rate,d_root,height,taxonomy_name,loss_decay,hierarchical_loss
0,0,0,f1_micro,0.897222,protonet,baseline_v0,32,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-0.5,False
1,1,0,f1_macro,0.896079,protonet,baseline_v0,32,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-0.5,False
2,2,0,hlca-mistake,1.626126,protonet,baseline_v0,32,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-0.5,False
3,3,0,f1_micro,0.850926,ancestor-proto,baseline_v0,32,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-0.5,False
4,4,0,f1_macro,0.854191,ancestor-proto,baseline_v0,32,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-0.5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,495,99,f1_micro,0.553704,protonet,hproto_v2,1,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-1.0,True
8996,496,99,f1_macro,0.568818,protonet,hproto_v2,1,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-1.0,True
8997,497,99,hlca-mistake,1.530083,protonet,hproto_v2,1,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-1.0,True
8998,498,99,f1_micro,0.738889,ancestor-proto,hproto_v2,1,12,True,True,...,1,20,12,hprotonet,0.03,128,1,joint-taxonomy,-1.0,True


# Results Table

In [151]:
import numpy as np
metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = list(reversed(df['n_shot'].unique()))
models = df['model'].unique()

results = {}
for tag in tags:
    results[tag] = []
    for n_shot in shots:
        for model in models:
            if model == 'hproto_v2': continue
                
            ms = {
                    'model': model,
                    'n_shot': n_shot, 
            }
            for metric in metrics:
                values = np.array(df[(df['metric'] == metric) & \
                            (df['tag'] == tag) & \
                            (df['n_shot'] == n_shot) & \
                            (df['model'] == model) 
                           ]['value'].values)
                
                ms[metric] = f'{values.mean():.4f} ± {values.std():.4f}'
            
            results[tag].append(ms)
            

print(f"{pd.DataFrame(results['protonet'], index=None).to_markdown()}")

|    | model       |   n_shot | f1_micro        | f1_macro        | hlca-mistake     |
|---:|:------------|---------:|:----------------|:----------------|:-----------------|
|  0 | baseline_v0 |        1 | 6.2868 ± 0.7486 | 6.0614 ± 0.7890 | 16.5235 ± 1.2045 |
|  1 | hproto_v1   |        1 | 6.3796 ± 0.6823 | 6.1800 ± 0.7035 | 16.2366 ± 1.2477 |
|  2 | baseline_v0 |        2 | 7.0122 ± 0.6372 | 6.8592 ± 0.7044 | 16.3745 ± 1.3941 |
|  3 | hproto_v1   |        2 | 7.2365 ± 0.5537 | 7.0945 ± 0.5837 | 16.0504 ± 1.5334 |
|  4 | baseline_v0 |        4 | 7.6103 ± 0.5922 | 7.4786 ± 0.6624 | 16.2548 ± 1.3657 |
|  5 | hproto_v1   |        4 | 7.8325 ± 0.4981 | 7.7275 ± 0.5486 | 15.7924 ± 1.3882 |
|  6 | baseline_v0 |        8 | 7.9001 ± 0.6354 | 7.7952 ± 0.7110 | 16.2174 ± 1.4633 |
|  7 | hproto_v1   |        8 | 8.1802 ± 0.4557 | 8.1051 ± 0.4934 | 15.5564 ± 1.6121 |
|  8 | baseline_v0 |       16 | 8.2983 ± 0.4862 | 8.2576 ± 0.5176 | 16.1771 ± 1.5391 |
|  9 | hproto_v1   |       16 | 8.4618 ± 0.

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning:

Mean of empty slice.

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/home/hugo/venv/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning:

Degrees of freedom <= 0 for slice

/

# Boxplots and Statistical Significance

In [155]:
from scipy.stats import wilcoxon

def significance(df, pairs):
    
    for pair in pairs:
        (s1, m1), (s2, m2) = pair

        df1 = df[(df['n_shot'] == s1) & (df['model'] == m1)].copy()
        df2 = df[(df['n_shot'] == s2) & (df['model'] == m2)].copy()

        stat, p = wilcoxon(df1['value'].values, df2['value'].values)
        
        print(f'\t{pair} --> p < {p}')

In [156]:
import plotly.express as px
import pandas as pd
from itertools import product

metrics = df['metric'].unique()
tags = df['tag'].unique()
shots = df['n_shot'].unique()
models = df['model'].unique()
print(models)

figs = []
for tag in tags:
    for metric in metrics:
        subdf = df
        subdf = df[(df['metric'] == metric) & (df['tag'] == tag)].copy()
        
        # reverse it
        subdf = subdf.astype({'n_shot': 'str'})
        subdf = subdf.iloc[::-1]
        
        pairs = [((str(s1), m1), (str(s2), m2)) for s1, s2, m1, m2 \
                 in product(shots, shots, models, models) \
                    if s1 == s2 and m1 != m2 and 'hproto_v2' not in m1 and 'hproto_v2' not in m2]
        
        print(f'{tag}-{metric}')
        significance(subdf, pairs)

        if len(subdf) == 0:
            continue
        
        range_y = [0, 1] if 'f1' in metric else None
        
        fig = px.box(subdf, x='n_shot', y='value', color='model',\
                     title=f'{tag}-{metric}', range_y=range_y)
        
        figs.append({
            'tag': tag, 
            'metric': metric, 
            'fig': fig
        })

metrics

['baseline_v0' 'hproto_v1' 'hproto_v2']
protonet-f1_micro
	(('32', 'baseline_v0'), ('32', 'hproto_v1')) --> p < 0.997215095836184
	(('32', 'hproto_v1'), ('32', 'baseline_v0')) --> p < 0.997215095836184
	(('16', 'baseline_v0'), ('16', 'hproto_v1')) --> p < 6.336449755468122e-05
	(('16', 'hproto_v1'), ('16', 'baseline_v0')) --> p < 6.336449755468122e-05
	(('8', 'baseline_v0'), ('8', 'hproto_v1')) --> p < 2.3681920180621767e-07
	(('8', 'hproto_v1'), ('8', 'baseline_v0')) --> p < 2.3681920180621767e-07
	(('4', 'baseline_v0'), ('4', 'hproto_v1')) --> p < 1.1849589500904755e-05
	(('4', 'hproto_v1'), ('4', 'baseline_v0')) --> p < 1.1849589500904755e-05
	(('2', 'baseline_v0'), ('2', 'hproto_v1')) --> p < 2.669836858935779e-05
	(('2', 'hproto_v1'), ('2', 'baseline_v0')) --> p < 2.669836858935779e-05
	(('1', 'baseline_v0'), ('1', 'hproto_v1')) --> p < 0.1606621884044236
	(('1', 'hproto_v1'), ('1', 'baseline_v0')) --> p < 0.1606621884044236
protonet-f1_macro
	(('32', 'baseline_v0'), ('32', 'hprot

ValueError: The exact distribution of the Wilcoxon test statistic is not implemented for n=0

In [122]:
figs[0]['fig']

In [123]:
figs[1]['fig']

In [124]:
figs[2]['fig']

In [125]:
figs[3]['fig']

In [45]:
f = open('/home/aldo/documents/hierachical-few-shot/music-trees/music_trees/assets/taxonomies/origin-taxonomy.yaml')

f = f.read().split('\n')

In [59]:
countries = []

for instrument in f:
    if not instrument.split('#')[1][1:] in countries:
        countries.append(instrument.split('#')[1][1:])
        
ins_origins = {country:[] for country in countries}



In [60]:
for line in f:
    split_line = line.split('#')
    country = split_line[1][1:] 
    instrument = split_line[0]
    ins_origins[country].append(instrument)

In [64]:
for key in ins_origins.keys():
    print(key, ':')
    for ins in ins_origins[key]:
        print('\t', ins)

China :
	 - erhu 
	 - guzheng 
	 - liuqin 
	 - zhongruan 
	 - yangqin 
	 - chimes 
	 - cowbell 
	 - gong 
	 - gu 
	 - toms 
Italy :
	 - violin 
	 - viola 
	 - cello 
	 - double bass 
	 - violin section 
	 - viola section 
	 - cello section 
	 - string section 
	 - harpsichord 
	 - mandolin 
	 - tack piano 
	 - bassoon 
	 - melodica 
	 - electric piano 
India :
	 - dilruba 
	 - sitar 
	 - harmonium 
United States :
	 - acoustic guitar 
	 - clean electric guitar 
	 - electric bass 
	 - banjo 
	 - slide guitar 
	 - dulcimer 
	 - male rapper 
	 - female rapper 
	 - beatboxing 
	 - vibraphone 
	 - high hat 
Hawaii :
	 - lap steel guitar 
	 - ukulele 
Egypt :
	 - harp 
	 - bagpipe 
	 - trumpet 
	 - triangle 
	 - rattle 
	 - darbuka 
	 - doumbek 
Middle East :
	 - oud 
Italy  :
	 - piano 
France :
	 - dizi 
	 - oboe 
	 - cornet 
Germany :
	 - flute 
	 - flute section 
	 - piccolo 
	 - recorder 
	 - alto saxophone 
	 - bass clarinet 
	 - clarinet 
	 - tenor saxophone 
	 - soprano saxophone 
	 

In [3]:
import wikipedia 

In [9]:
wikipedia.search("pages")

['List of countries and dependencies by area',
 'List of African countries by area',
 'List of countries by GDP (PPP) per capita',
 'List of countries by GDP (nominal) per capita',
 'List of countries by income equality',
 'List of countries by GDP (PPP)',
 'List of countries by Human Development Index',
 'List of European countries by area',
 'List of countries and dependencies by population density',
 'List of countries by military expenditures']

In [7]:
wikipedia.page(f[0][1:]).content

'The erhu (Chinese: 二胡; pinyin: èrhú; [aɻ˥˩xu˧˥]), is a two-stringed bowed musical instrument, more specifically a spike fiddle, which may also be called a Southern Fiddle, and is sometimes known in the Western world as the Chinese violin or a Chinese two-stringed fiddle.\nIt is used as a solo instrument as well as in small ensembles and large orchestras. It is the most popular of the huqin family of traditional bowed string instruments used by various ethnic groups of China. As a very versatile instrument, the erhu is used in both traditional and contemporary music arrangements, such as in pop, rock and jazz.\n\n\n== History ==\n\nThe Erhu can be traced back to proto-Mongolic instruments which first appeared in China during the Tang Dynasty. It is believed to have evolved from the Xiqin (奚 琴). The xiqin is believed to have originated from the Xi people located in current northeast China.\nThe first Chinese character of the name of the instrument (二, èr, two) is believed to come from t